<a href="https://www.kaggle.com/code/peremartramanonellas/rouge-evaluation-untrained-vs-trained-llm?scriptVersionId=162222822" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# ROUGE Metrics
The metrics that we have been using up to now with more traditional models, such as Accuracy, F1 Score or Recall, do not help us to evaluate the results of generative models.

With these models, we are beginning to use metrics such as BLEU, ROUGE, or METEOR. Metrics that are adapted to the objective of the model.

ROUGE consists of a set of metrics that are used to compare the quality of a generated text against a reference text.

It can be used whenever we have a reference text available. Some of the most common applications are: translations, text summarization, or entity extraction between others.

It’s important to note that ROUGE is only as reliable as the quality of the reference text. If they have poor quality, the results provided by ROUGE may not reflect correctly the quality of the generated text.

In this notebook, I want to explain how to use the ROUGE metrics to measure the quality of the summaries produced by the newest large language models.

If you're looking for more detailed explanations, you can refer to the article: [Rouge Metrics: Evaluating Summaries in Large Language Models](https://medium.com/towards-artificial-intelligence/rouge-metrics-evaluating-summaries-in-large-language-models-d200ee7ca0e6)

# Load the Data

In this first step, we will use two versions of the same model to generate summaries for the news in the dataset. One of the versions is fine-tuned specifically for summarization.

In [1]:
#Import generic libraries
import numpy as np 
import pandas as pd
import torch


The dataset is available on Kaggle and comprises a collection of technological news articles compiled by MIT. The article text is located in the 'Article Body' column.

https://www.kaggle.com/datasets/deepanshudalal09/mit-ai-news-published-till-2023

In [2]:
news = pd.read_csv('/kaggle/input/mit-ai-news-published-till-2023/articles.csv')
DOCUMENT="Article Body"

In [3]:
#Because it is just a sample notebook we select a small portion of News.
MAX_NEWS = 10
subset_news = news.head(MAX_NEWS)

In [4]:
subset_news.head(2)

,Unnamed: 0,Published Date,Author,Source,Article Header,Sub_Headings,Article Body,Url
0,0,"July 7, 2023",Adam Zewe,MIT News Office,Learning the language of molecules to predict ...,This AI system only needs a small amount of da...,['Discovering new materials and drugs typicall...,https://news.mit.edu/2023/learning-language-mo...
1,1,"July 6, 2023",Alex Ouyang,Abdul Latif Jameel Clinic for Machine Learning...,MIT scientists build a system that can generat...,"BioAutoMATED, an open-source, automated machin...",['Is it possible to build machine-learning mod...,https://news.mit.edu/2023/bioautomated-open-so...


In [5]:
#recovering the articles to summarize
articles = subset_news[DOCUMENT].tolist()

# Load the Models and create the summaries

Both models are available on Hugging Face, so we will work with the Transformers library.

In [6]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#T5 Base
model_name_small = "t5-base"
#T5 FineTuned for summarisation
model_name_reference = "flax-community/t5-base-cnn-dm"
#model_name_reference = "pszemraj/long-t5-tglobal-base-16384-booksum-V11-big_patent-V2"

In [7]:
#This function returns the tokenizer and the Model. 
def get_model(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    
    return tokenizer, model
    

In [8]:
tokenizer_small, model_small = get_model(model_name_small)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxver

In [9]:
tokenizer_reference, model_reference = get_model(model_name_reference)

With both models downloaded and ready, we create a function that will perform the summaries.

The function takes fourth parameters:

* the list of texts to summarize.
* the tokenizer.
* the model.
* the maximum length for the generated summary

In [10]:
def create_summaries(texts_list, tokenizer, model, max_l=125):
    
    # We are going to add a prefix to each article to be summarized 
    # so that the model knows what it should do
    prefix = "Summarize this news: "  
    summaries_list = [] #Will contain all summaries

    texts_list = [prefix + text for text in texts_list]
    
    for text in texts_list:
        
        summary=""
        
        #calculate the encodings
        input_encodings = tokenizer(text, 
                                    max_length=1024, 
                                    return_tensors='pt', 
                                    padding=True, 
                                    truncation=True)

        # Generate summaries
        with torch.no_grad():
            output = model.generate(
                input_ids=input_encodings.input_ids,
                attention_mask=input_encodings.attention_mask,
                max_length=max_l,  # Set the maximum length of the generated summary
                num_beams=2,     # Set the number of beams for beam search
                early_stopping=True
            )
            
        #Decode to get the text
        summary = tokenizer.batch_decode(output, skip_special_tokens=True)
        
        #Add the summary to summaries list 
        summaries_list += summary
    return summaries_list 
    

To create the summaries, we call the 'create_summaries' function, passing both the news articles and the corresponding tokenizer and model.

In [11]:
# Creating the summaries for both models. 
summaries_small = create_summaries(articles, 
                                  tokenizer_small, 
                                  model_small)


In [12]:
summaries_reference = create_summaries(articles, 
                                      tokenizer_reference, 
                                      model_reference)

In [13]:
summaries_small

['MIT and MIT-Watson AI Lab have developed a unified framework. the system can simultaneously predict molecular properties and generate new molecules. it uses this grammar to construct viable molecules and predict their properties.',
 '\'BioAutoMATED\' is an automated machine-learning system that can select and build an appropriate model for a given dataset. it can even take care of the laborious task of data preprocessing, whittling down a months-long process to just a few hours. \'"We want to lower these barriers for a lot of folks that want to use machine learning or biology," says first co-author Jacqueline Valeri.',
 "MIT and IBM research scientists have made a computer vision model more robust by training it to work like a part of the brain that humans and other primates rely on for object recognition. 'we asked the artificial neural network to make the function of one of your inside simulated “neural” layers as similar as possible to the corresponding biological neural layer,' s

In [14]:
summaries_reference

['Researchers created a machine-learning system that automatically learns the "language" of molecules using only a small, domain-specific dataset. The system learns to construct viable molecules and predict their properties. Computational design and Fabrication Group will be presented at the International Conference for Machine Learning.',
 "Automated machine-learning system can select and build an appropriate model for a given dataset. 'BioAutoMATED' is an automated machine-learning system. The tool includes binary classification models, multi-class classification models, and more complex neural networks.",
 "MIT and IBM researchers have found that artificial neural networks resemble the multilayered brain circuits that process visual information in humans and other primates. 'We asked it to do both of those things as well as the standard, computer vision approach,' said one expert. The network found to be more robust by training it to work like a part of the brain that humans rely on

At first glance, it's evident that the summaries are different. 

However, it's challenging to determine which one is better. 

It's even difficult to discern whether they are significantly distinct or if there are just subtle differences between them.

This is what we are going to verify now using ROUGE. When comparing the summaries of one model with those of the other, we don't get an idea of which one is better, but rather an idea of how much the summaries have changed with the fine-tuning applied to the model.

# ROUGE
Let's install and load all the necessary libraries to conduct a ROUGE evaluation.

In [15]:
!pip install evaluate
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=3b3e25ad62cd6fe3b7b6c50696b72affb0feb0d049359a50192e86ca5a3338b8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be

In [16]:
import evaluate
from nltk.tokenize import sent_tokenize

#!pip install rouge

In [17]:
#import evaluate
#from nltk.tokenize import sent_tokenize
#from rouge_score import rouge_scorer

In [18]:
#With the function load of the library evaluate 
#we create a rouge_score object
rouge_score = evaluate.load("rouge")

Calculating ROUGE is as simple as calling the *compute* function of the *rouge_score* object we created earlier. This function takes the texts to compare as arguments and a third value *use_stemmer*, which indicates whether it should use *stemmer* or full words for the comparison.

A *stemmer* is the base of the word. Transform differents forms of a word in a same base. 

Some samples of steammer are: 
* Jumping -> Jump. 
* Running -> Run. 
* Cats -> Cat. 

In [19]:
def compute_rouge_score(generated, reference):
    
    #We need to add '\n' to each line before send it to ROUGE
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]
    
    return rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,
        
    )

In [20]:
compute_rouge_score(summaries_small, summaries_reference)

{'rouge1': 0.37533050121921296,
 'rouge2': 0.19971526600740083,
 'rougeL': 0.28093139506008846,
 'rougeLsum': 0.3404713030404797}

We can see that there is a difference between the two models when performing summarization. 

For example, in ROUGE-1, the similarity is 47%, while in ROUGE-2, it's a 32%. This indicates that the results are different, with some similarities but differents enough. 

However, we still don't know which model is better since we have compared them to each other and not to a reference text. But at the very least, we know that the fine-tuning process applied to the second model has significantly altered its results.

# Comparing to a Dataset with real summaries. 
We are going to load the Dataset cnn_dailymail. This is a well-known dataset available in the **Datasets** library, and it suits our purpose perfectly. 

Apart from the news, it also contains pre-existing summaries. 

We will compare the summaries generated by the two models we are using with those from the dataset to determine which model creates summaries that are closer to the reference ones.

In [21]:
!pip install --upgrade datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 26.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [22]:
from datasets import load_dataset

cnn_dataset = load_dataset(
    "ccdv/cnn_dailymail", version="3.0.0"
)


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___cnn_dailymail/default/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
#Get just a few news to test
sample_cnn = cnn_dataset["test"].select(range(MAX_NEWS))

sample_cnn

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 10
})

We retrieve the maximum length of the summaries to give the models the option to generate summaries of the same length, if they choose to do so.

In [24]:
max_length = max(len(item['highlights']) for item in sample_cnn)
max_length = max_length + 10

In [25]:
summaries_t5_base = create_summaries(sample_cnn["article"], 
                                      tokenizer_small, 
                                      model_small, 
                                      max_l=max_length)

In [26]:
summaries_t5_finetuned = create_summaries(sample_cnn["article"], 
                                      tokenizer_reference, 
                                      model_reference, 
                                      max_l=max_length)

In [27]:
#Get the real summaries from the cnn_dataset
real_summaries = sample_cnn['highlights']

Let's take a look at the generated summaries alongside the reference summaries provided by the dataset.

In [28]:
summaries = pd.DataFrame.from_dict(
        {
            "base": summaries_t5_base, 
            "finetuned": summaries_t5_finetuned,
            "reference": real_summaries,
        }
    )
summaries.head()

,base,finetuned,reference
0,"best died in hospice in Hickory, north Carolin...","Jimmie Best was ""the most constantly creative ...","James Best, who played the sheriff on ""The Duk..."
1,"""it doesn't matter what anyone says, he is pre...",Dr. Anthony Moschetto's attorney calls the all...,A lawyer for Dr. Anthony Moschetto says the ch...
2,president Barack Obama took part in a roundtab...,President Obama says climate change is a publi...,"""No challenge poses more of a public threat th..."
3,independent channel in Russia aired Hillary Cl...,A Russian TV channel aired Hillary Clinton's f...,"Presidential hopeful's video, featuring gay co..."
4,sen. marco rubio is running for the 2016 presi...,Ruben Navarrette: Marco Rubio is missing a cha...,"Raul Reyes: In seeking Latino vote, Marco Rubi..."


In [29]:
pd.set_option('display.max_colwidth', None)
summaries.head(2)

,base,finetuned,reference
0,"best died in hospice in Hickory, north Carolina, of complications from pneumonia. he played bumbling sheriff Rosco P. Coltrane on ""the Dukes of Hazzard"" he was born in Kentucky and raised in rural Indiana.","Jimmie Best was ""the most constantly creative person I have ever known,"" co-star John Schneider says. ""The Dukes of Hazzard"" ran until 1985 and spawned TV movies, an animated series and video games. ""Jimmie Best was my teacher, mentor, close friend and collaborator for 26 years,"" friend says.","James Best, who played the sheriff on ""The Dukes of Hazzard,"" died Monday at 88 .\n""Hazzard"" ran from 1979 to 1985 and was among the most popular shows on TV ."
1,"""it doesn't matter what anyone says, he is presumed to be innocent,"" attorney says. cardiologist's lawyer says allegations against his client are ""completely unsubstantiated"" prosecutors say he pleaded not guilty to all charges. he faces charges in connection with a plot to take out a rival doctor.","Dr. Anthony Moschetto's attorney calls the allegations ""completely unsubstantiated"" He is accused of plotting to kill a rival doctor on Long Island.","A lawyer for Dr. Anthony Moschetto says the charges against him are baseless .\nMoschetto, 54, was arrested for selling drugs and weapons, prosecutors say .\nAuthorities allege Moschetto hired accomplices to burn down the practice of former associate ."


Now we can calculate the ROUGE scores for the two models.

In [30]:
compute_rouge_score(summaries_t5_base, real_summaries)

{'rouge1': 0.3953057068328145,
 'rouge2': 0.16837914761792147,
 'rougeL': 0.2946575626622585,
 'rougeLsum': 0.3565254365696814}

In [31]:
compute_rouge_score(summaries_t5_finetuned, real_summaries)

{'rouge1': 0.3394672308099006,
 'rouge2': 0.11825310327497204,
 'rougeL': 0.2493055504256188,
 'rougeLsum': 0.30606197407216174}

With these results, I would say that the fine-tuned model performs slightly better than the T5-Base model. It consistently achieves higher ROUGE scores in all metrics except for LSUM, where the difference is minimal.

Additionally, the ROUGE metrics are quite interpretable. 

LSUM indicates the percentage of the longest common subsequence, regardless of word order, in relation to the total length of the text. 

This can be a good indicator of overall similarity between texts. However, both models have very similar LSUM scores, and the fine-tuned model has better scores in other ROUGE metrics.

Personally, I would lean towards the fine-tuned model, although the difference may not be very significant.

## Continue learning
This notebook is part of a [course on large language models](https://github.com/peremartra/Large-Language-Model-Notebooks-Course) I'm working on and it's available on [GitHub](https://github.com/peremartra/Large-Language-Model-Notebooks-Course). You can see the other lessons and if you like it, don't forget to subscribe to receive notifications of new lessons.

Other notebooks in the Large Language Models series: 
https://www.kaggle.com/code/peremartramanonellas/use-a-vectorial-db-to-optimize-prompts-for-llms
https://www.kaggle.com/code/peremartramanonellas/ask-your-documents-with-langchain-vectordb-hf


### Feel Free to fork or edit the noteboook for you own convenience. Please consider ***UPVOTING IT***. It helps others to discover the notebook, and it encourages me to continue publishing.